In [82]:
import pandas as pd
import numpy as np

In [83]:
df = pd.read_csv("SPAM text message 20170820 - Data.csv")

In [84]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [85]:
df['Category'] = df['Category'].map({'ham':0,'spam':1})

In [86]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [88]:
# Remove spacial Characters
import re

df['Message'] = df['Message'].apply(lambda x :re.sub("[^a-zA-Z \s]",'',x)) # Remove Special Characters

df['Message'] = df['Message'].apply(lambda x :re.sub('\[.*?\]','',x)) # Remove Special Characters

df['Message'] = df['Message'].apply(lambda x :re.sub('https?://\S+|www\.\S+','',x)) # Remove Special Characters

df['Message'] = df['Message'].apply(lambda x :re.sub('<.*?>+','',x)) # Remove Special Characters

df['Message'] = df['Message'].apply(lambda x :re.sub('\n','',x)) # Remove Special Characters

df['Message'] = df['Message'].apply(lambda x :re.sub('\w*\d\w*','',x)) # Remove Special Characters


df['Message'] = [x.lower() for x in df['Message']]

from nltk.tokenize import word_tokenize
df['Message'] = [word_tokenize(x) for x in df['Message']]


In [89]:
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()

df['Message'] = [' '.join([lemma.lemmatize(y) for y in x]) for x in df["Message"]]

In [99]:
# Count Vectorizer

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features = 100, stop_words = "english")

sparce_matrix = count_vectorizer.fit_transform(df['Message']).toarray()
tokens = count_vectorizer.get_feature_names()

In [103]:
sparce_matrix[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [106]:
len(tokens)

100

In [108]:
pd.DataFrame(sparce_matrix, columns=tokens).head()

,amp,ask,babe,buy,care,cash,claim,come,contact,da,...,want,wat,way,week,win,won,work,yeah,year,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
# Tf-Idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=100)
tfidfmatrix = vectorizer.fit_transform(df['Message']).toarray()

cname = vectorizer.get_feature_names()

In [116]:
pd.DataFrame(tfidfmatrix,columns=cname).head()

,about,all,am,an,and,any,are,at,back,be,...,want,we,week,well,what,when,will,with,you,your
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# N Gram

count_vectorizer = CountVectorizer(max_features = 100, stop_words = "english",ngram_range=(2, 2),)
sparce_matrix = count_vectorizer.fit_transform(df["Message"]).toarray()
tokens = count_vectorizer.get_feature_names()
gram2 = pd.DataFrame(sparce_matrix, columns=tokens)
gram2.head()

,account statement,attempt contact,await collection,caller prize,camcorder reply,camera phone,cash prize,chance win,claim code,claim ur,...,urgent mobile,valid hr,wan na,want come,want new,wat doing,wat time,week just,won guaranteed,won prize
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
# Text Classification
df.head()

,Category,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final ...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he go to usf he life around h...


In [121]:
from sklearn.model_selection import train_test_split

y = df.iloc[:,0].values   
x = sparce_matrix
tfidfx = tfidfmatrix

In [124]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 2019)

In [125]:
tf_x_train, tf_x_test, tf_y_train, tf_y_test = train_test_split(tfidfmatrix ,y,test_size = 0.3,random_state = 2019)

In [128]:
gm_x_train, gm_x_test, gm_y_train, gm_y_test = train_test_split(gram2 ,y,test_size = 0.3,random_state = 2019)

In [131]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 

from sklearn.naive_bayes import GaussianNB

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)
print('CountVectorizer Accuracy Score',nb.score(x_test,y_test))

nb.fit(tf_x_train, tf_y_train)
print('TF-IDF Vectorizer Accuracy Score',nb.score(tf_x_test,tf_y_test))

nb.fit(gm_x_train, gm_y_train)
print('bi-gram Vectorizer Accuracy Score',nb.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.5149521531100478
TF-IDF Vectorizer Accuracy Score 0.6830143540669856
bi-gram Vectorizer Accuracy Score 0.937200956937799


In [132]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

nb.fit(x_train, y_train)
print('CountVectorizer Accuracy Score',nb.score(x_test,y_test))

nb.fit(tf_x_train, tf_y_train)
print('TF-IDF Vectorizer Accuracy Score',nb.score(tf_x_test,tf_y_test))

nb.fit(gm_x_train, gm_y_train)
print('bi-gram Vectorizer Accuracy Score',nb.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.9599282296650717
TF-IDF Vectorizer Accuracy Score 0.9431818181818182
bi-gram Vectorizer Accuracy Score 0.9360047846889952


In [133]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()

nb.fit(x_train, y_train)
print('CountVectorizer Accuracy Score',nb.score(x_test,y_test))

nb.fit(tf_x_train, tf_y_train)
print('TF-IDF Vectorizer Accuracy Score',nb.score(tf_x_test,tf_y_test))

nb.fit(gm_x_train, gm_y_train)
print('bi-gram Vectorizer Accuracy Score',nb.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.9641148325358851
TF-IDF Vectorizer Accuracy Score 0.9641148325358851
bi-gram Vectorizer Accuracy Score 0.9366028708133971


In [134]:
# SVM

from sklearn import svm
svmmodel = svm.SVC(kernel='linear', C = 1)
svmmodel.fit(x_train, y_train)

print('CountVectorizer Accuracy Score',svmmodel.score(x_test,y_test))
svmmodel.fit(tf_x_train, tf_y_train)

print('TF-IDF Vectorizer Accuracy Score',svmmodel.score(tf_x_test,tf_y_test))
svmmodel.fit(gm_x_train, gm_y_train)

print('bi-gram Vectorizer Accuracy Score',svmmodel.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.9581339712918661
TF-IDF Vectorizer Accuracy Score 0.9677033492822966
bi-gram Vectorizer Accuracy Score 0.9354066985645934


In [135]:
from sklearn import svm
svmmodel = svm.SVC(kernel='linear', C = 1)
svmmodel.fit(tf_x_train, tf_y_train)

print('TF-IDF Vectorizer Accuracy Score',svmmodel.score(tf_x_test,tf_y_test))

TF-IDF Vectorizer Accuracy Score 0.9677033492822966


In [136]:
# LogisticRegression

from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=0, solver='lbfgs')
logit.fit(x_train, y_train)

print('CountVectorizer Accuracy Score',logit.score(x_test,y_test))
svmmodel.fit(tf_x_train, tf_y_train)

print('TF-IDF Vectorizer Accuracy Score',logit.score(tf_x_test,tf_y_test))
svmmodel.fit(gm_x_train, gm_y_train)

print('bi-gram Vectorizer Accuracy Score',logit.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.958732057416268
TF-IDF Vectorizer Accuracy Score 0.8552631578947368
bi-gram Vectorizer Accuracy Score 0.8755980861244019


In [137]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=50)
clf.fit(x_train, y_train)

print('CountVectorizer Accuracy Score',clf.score(x_test,y_test))
svmmodel.fit(tf_x_train, tf_y_train)

print('TF-IDF Vectorizer Accuracy Score',clf.score(tf_x_test,tf_y_test))
svmmodel.fit(gm_x_train, gm_y_train)

print('bi-gram Vectorizer Accuracy Score',clf.score(gm_x_test,gm_y_test))

CountVectorizer Accuracy Score 0.9479665071770335
TF-IDF Vectorizer Accuracy Score 0.8415071770334929
bi-gram Vectorizer Accuracy Score 0.8773923444976076
